In [1]:
import modin.pandas as pd
import numpy as np
import sys
sys.path.append('../')
from utils.stock_utils import *

提示：当前环境pandas版本为0.25，get_price与get_fundamentals_continuously接口panel参数将固定为False
注意：0.25以上版本pandas不支持panel，如使用该数据结构和相关函数请注意修改


In [2]:
from utils.data_source import *
from utils.env_constants import *

stocks = stock_all_jq[stock_all_jq.status == 'OK']
stk_code_jq = stocks.index.to_list()

auth success 
Work on data from 2018-01-01 to 2020-10-23
Constants:
stock_list(3541): ['000001.SZ', '000002.SZ']
stock_list_jq(3541): ['000001.XSHE', '000002.XSHE']
stock_list_all_jq(4158)
stock_all_jq(4158):
            display_name  name start_date   end_date   type status  not_new
000001.XSHE         平安银行  PAYX 1991-04-03 2200-01-01  stock     OK     True
000002.XSHE          万科A   WKA 1991-01-29 2200-01-01  stock     OK     True


In [3]:
import math
import requests as r
import json
from time import sleep
from tqdm.auto import trange, tqdm

SINA_DD_LIST = 'http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/CN_Bill.GetBillList?num=%s&page=%s&sort=ticktime&asc=1&volume=%s&amount=%s&type=0&day=%s'
SINA_DD_COUNT = 'http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/CN_Bill.GetBillListCount?sort=ticktime&asc=1&volume=%s&amount=%s&type=0&day=%s'
SINA_DD = 'http://vip.stock.finance.sina.com.cn/quotes_service/view/cn_bill_download.php?symbol=%s&num=%s&page=%s&sort=ticktime&asc=1&volume=0&amount=%s&type=0&day=%s'
PAGE_SIZE = 100
AMOUNT_BIG = 200000
VOL_BIG = 50000

page = 1

#         net_amount_xl	超大单净额(万)	超大单：大于等于50万股或者100万元的成交单
#         net_pct_xl	超大单净占比(%)	超大单净占比 = 超大单净额 / 成交额
#         net_amount_l	大单净额(万)	大单：大于等于10万股或者20万元且小于50万股或者100万元的成交单
#         net_pct_l	大单净占比(%)	大单净占比 = 大单净额 / 成交额
#         net_amount_m	中单净额(万)	中单：大于等于2万股或者4万元且小于10万股或者20万元的成交单
#         net_pct_m	中单净占比(%)	中单净占比 = 中单净额 / 成交额
#         net_amount_s	小单净额(万)	小单：小于2万股或者4万元的成交
#         net_pct_s	小单净占比(%)	小单净占比 = 小单净额 / 成交额

def get_page(dd_url, date):
    res = r.get(dd_url)
    df = pd.DataFrame(json.loads(res.content.decode('utf-8')))
    df['ts_code'] = df.symbol.apply(lambda x: x[-6:] + '.' + x[0:2].upper())
    df['time'] = df.ticktime.apply(lambda x: pdl.parse(date + 'T' + x))
    df.drop(columns=['ticktime', 'symbol'], inplace=True)
    df.set_index(['ts_code', 'time'], inplace=True)
    df = df.astype({'price': 'float', 'volume': 'int'})
    df['amount'] = df.price * df.volume
    return df


def watching_dd(date, amount=AMOUNT_BIG, volume=VOL_BIG, verbose=False):
    count_url = SINA_DD_COUNT % (volume, amount, date)
    if verbose:
        print(count_url)
    
    init_dd_count = int(r.get(count_url).content.decode('utf-8').replace('"', ''))
    
    page_size = 1000 if init_dd_count > 1000 else 100
    total_page_num = math.ceil(init_dd_count/page_size)
    
    if verbose:
        print(f'Total count {init_dd_count}, page size: {page_size}, page num: {total_page_num}.')
        
    df = pd.DataFrame(columns=['ts_code', 'time', 'name', 'price', 'volume', 'prev_price', 'kind', 'amount'])
    df.set_index(['ts_code', 'time'], inplace=True)
    
    tmp_dups = pd.DataFrame()
    dups = pd.DataFrame()
    
    for i in trange(1, total_page_num):
        dd_url = SINA_DD_LIST % (page_size, i+1, volume, amount, date)
        tmpdf = get_page(dd_url, date)
        tmpdf_dup = tmpdf[tmpdf.index.duplicated()]
        
        if len(tmpdf_dup) > 0:
            print(f'Batch {i} has {len(tmpdf_dup)} duplicates!')
            print(tmpdf_dup)
        
        df = df.append(tmpdf[~tmpdf.index.duplicated(keep='first')]) # most performant way to dedup index
        
        df_dup = df[df.index.duplicated()]
        if len(df_dup) > 0:
            print(f'Batch {i} has {len(df_dup)} duplicates with previous!')
            print(df_dup)
            df = df[~df.index.duplicated(keep='first')]
            
        sleep(0.1)
    
    print(f'Got {len(df)} records. Start monitoring updates...')
    
    # Monitoring starts, using 100 as page size now
    page_size = 100
    prev_page = math.ceil(init_dd_count/page_size)
    
#     prev_count = init_dd_count
    while pdl.now() < pdl.today().add(hours=15): # keep running til close market
#         current_count = int(r.get(count_url).content.decode('utf-8').replace('"', ''))
#         if current_count > prev_count:
        
        tmpdf = pd.DataFrame()       
        while len(tmpdf) < page_size:
            dd_url = SINA_DD_LIST % (page_size, prev_page, volume, amount, date)
            tmpdf = get_page(dd_url, date)
            if len(tmpdf) > 0:            
                df = pandas.concat([df,tmpdf]).drop_duplicates()
                # UPDATE Charts or xxx
        prev_page += 1
    
    return df

In [4]:
df = watching_dd(end_date, verbose=True)
df

http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/CN_Bill.GetBillListCount?sort=ticktime&asc=1&volume=50000&amount=200000&type=0&day=2020-10-23
Total count 46267, page size: 1000, page num: 47.


AttributeError: 'DataFrame' object has no attribute 'symbol'

In [7]:
df=ts.get_sina_dd('600848', date='2020-10-23') #默认400手
df

In [12]:
from jqdatasdk import alpha191
alpha191.alpha_001('600848.XSHG', end_date=None,fq='pre')

Exception: get_alpha_191 属于付费模块，已为您自动开通7天试用权限，再次登录即可生效；如需购买正式权限请联系JQData管理员，微信号：JQData02

In [ ]:
wc = pd.read_csv('/Users/tzhu/Downloads/sina_bill_2020-09-22_sh601860.csv')
wc.columns = ['code', 'name', 'time', 'price', 'volume', 'prev_price', 'kind']
wc.loc[:,'amount'] = wc.volume * wc.price
wc

In [81]:
wc[wc.duplicated(keep=False, subset=['price', 'volume', 'prev_price', 'kind'])]

,code,name,time,price,volume,prev_price,kind,amount
9,sz000338,潍柴动力,09:30:27,15.90,23200,15.90,卖盘,368880.0
12,sz000338,潍柴动力,09:30:39,15.90,23200,15.90,卖盘,368880.0
128,sz000338,潍柴动力,09:39:24,15.83,22200,15.83,买盘,351426.0
361,sz000338,潍柴动力,10:15:12,15.82,15200,15.82,买盘,240464.0
364,sz000338,潍柴动力,10:15:33,15.82,15200,15.82,买盘,240464.0
...,...,...,...,...,...,...,...,...
571,sz000338,潍柴动力,13:17:33,15.83,22200,15.83,买盘,351426.0
794,sz000338,潍柴动力,14:29:12,15.64,16800,15.64,卖盘,262752.0
944,sz000338,潍柴动力,14:46:06,15.60,19300,15.60,买盘,301080.0
967,sz000338,潍柴动力,14:48:54,15.60,19300,15.60,买盘,301080.0


In [87]:
wc.tail(65).head(10)

,code,name,time,price,volume,prev_price,kind,amount
960,sz000338,潍柴动力,14:48:09,15.63,31800,15.63,买盘,497034.0
961,sz000338,潍柴动力,14:48:12,15.63,26500,15.63,买盘,414195.0
962,sz000338,潍柴动力,14:48:21,15.62,16200,15.62,卖盘,253044.0
963,sz000338,潍柴动力,14:48:24,15.60,23600,15.62,卖盘,368160.0
964,sz000338,潍柴动力,14:48:27,15.61,20800,15.60,卖盘,324688.0
965,sz000338,潍柴动力,14:48:45,15.60,15000,15.60,卖盘,234000.0
966,sz000338,潍柴动力,14:48:51,15.60,132600,15.61,卖盘,2068560.0
967,sz000338,潍柴动力,14:48:54,15.60,19300,15.60,买盘,301080.0
968,sz000338,潍柴动力,14:49:06,15.61,22500,15.59,买盘,351225.0
969,sz000338,潍柴动力,14:49:15,15.62,83600,15.61,买盘,1305832.0


In [90]:
df.loc['000338.SZ'].tail(60).head(10)

,name,price,volume,prev_price,kind,amount
time,,,,,,
2020-09-22 14:48:24+00:00,潍柴动力,15.60,23600,15.620,D,368160.00
2020-09-22 14:48:27+00:00,潍柴动力,15.61,20800,15.600,D,324688.00
2020-09-22 14:48:45+00:00,潍柴动力,15.60,15000,15.600,D,234000.00
2020-09-22 14:48:51+00:00,潍柴动力,15.60,132600,15.610,D,2068560.00
2020-09-22 14:49:06+00:00,潍柴动力,15.61,22500,15.590,U,351225.00
2020-09-22 14:49:15+00:00,潍柴动力,15.62,83600,15.610,U,1305832.00
2020-09-22 14:50:00+00:00,潍柴动力,15.64,16500,15.640,U,258060.00
2020-09-22 14:50:24+00:00,潍柴动力,15.64,18294,15.640,U,286118.16
2020-09-22 14:50:39+00:00,潍柴动力,15.65,37200,15.640,U,582180.00


In [79]:
wc.iloc[12]

code          sz000338
name              潍柴动力
time          09:30:39
price             15.9
volume           23200
prev_price        15.9
kind                卖盘
amount          368880
Name: 12, dtype: object

In [80]:
wc.iloc[9] == wc.iloc[12]

code           True
name           True
time          False
price          True
volume         True
prev_price     True
kind           True
amount         True
dtype: bool

In [71]:
df.loc['000338.SZ'].head(10)

,name,price,volume,prev_price,kind,amount
time,,,,,,
2020-09-22 09:30:00+00:00,潍柴动力,15.88,13600,15.850,U,215968.00
2020-09-22 09:30:03+00:00,潍柴动力,15.89,257599,15.880,U,4093248.11
2020-09-22 09:30:06+00:00,潍柴动力,15.90,83900,15.890,U,1334010.00
2020-09-22 09:30:09+00:00,潍柴动力,15.88,75500,15.900,E,1198940.00
2020-09-22 09:30:12+00:00,潍柴动力,15.88,30000,15.880,U,476400.00
2020-09-22 09:30:15+00:00,潍柴动力,15.90,23800,15.880,U,378420.00
2020-09-22 09:30:18+00:00,潍柴动力,15.89,29400,15.900,D,467166.00
2020-09-22 09:30:21+00:00,潍柴动力,15.92,23400,15.890,U,372528.00
2020-09-22 09:30:27+00:00,潍柴动力,15.90,23200,15.900,D,368880.00


In [95]:
tmpdf[tmpdf.index.isin(tmpdf.index.drop_duplicates())]

,,name,price,volume,prev_price,kind,amount
ts_code,time,,,,,,
603002.SH,2020-09-18 09:25:00+00:00,宏昌电子,5.06,49800,0.000,U,251988.0
603922.SH,2020-09-18 09:25:00+00:00,金鸿顺,18.57,45500,0.000,U,844935.0
601888.SH,2020-09-18 09:25:00+00:00,中国中免,210.00,101000,0.000,U,21210000.0
603956.SH,2020-09-18 09:25:00+00:00,威派格,20.01,38600,0.000,U,772386.0
601233.SH,2020-09-18 09:25:00+00:00,桐昆股份,15.12,121600,0.000,U,1838592.0
...,...,...,...,...,...,...,...
300399.SZ,2020-09-18 09:25:03+00:00,京天利,22.45,101390,0.000,D,2276205.5
300401.SZ,2020-09-18 09:25:03+00:00,花园生物,14.95,58700,0.000,D,877565.0
300404.SZ,2020-09-18 09:25:03+00:00,博济医药,12.69,38900,0.000,D,493641.0


In [42]:
dd_url = SINA_DD_LIST % (1000, 2, 200000, end_date)
tmpdf = get_page(dd_url)
tmpdf[tmpdf.index.isin(tmp_dups.index)]

,,name,price,volume,prev_price,kind,amount
ts_code,time,,,,,,


In [44]:
end_date

'2020-09-22'

In [65]:
# 1. original result cut by time
print(len(df))
df_dedup = df.drop_duplicates()
print(len(df_dedup))
tmp = df_dedup.groupby(['ts_code', 'kind']).agg({'volume': 'sum', 'amount': 'sum'})

798112
790950


In [5]:
# 2. get today price
# TODO: test if this returns today realtime vol & price
today_date = end_date
price_df = jq.get_price(security=stk_code_jq, start_date=today_date, end_date=today_date,frequency='daily', skip_paused=True)
price_df.loc[:,'time'] = datetime.now()
price_df.loc[:, 'ts_code'] = price_df.code.apply(get_ts_code)
price_df.rename(columns={'money': 'amount'}, inplace=True).set_index(['ts_code', 'time'], inplace=True)
price_df[['volume', 'amount']]

,time,code,open,close,high,low,volume,money
0,2020-09-22,002233.XSHE,14.91,14.82,15.05,14.82,9926497.0,1.480423e+08
1,2020-09-22,603809.XSHG,25.34,24.91,25.67,24.80,1687496.0,4.236285e+07
2,2020-09-22,002897.XSHE,34.36,34.29,34.90,33.90,1408079.0,4.843900e+07
3,2020-09-22,002906.XSHE,17.02,17.17,17.77,16.91,2954159.0,5.120960e+07
4,2020-09-22,002596.XSHE,6.47,6.43,6.52,6.43,6310042.0,4.085287e+07
...,...,...,...,...,...,...,...,...
3609,2020-09-22,603392.XSHG,179.00,187.01,193.88,179.00,2124774.0,4.000424e+08
3610,2020-09-22,601186.XSHG,8.73,8.65,8.76,8.65,22281704.0,1.938561e+08
3611,2020-09-22,603538.XSHG,40.18,39.68,40.76,39.50,1829782.0,7.334518e+07
3612,2020-09-22,601598.XSHG,3.89,3.89,3.95,3.83,39926334.0,1.553834e+08


In [20]:
tmp = ts.get_sina_dd(code, date='2020-04-17')
tt = ts.get_today_ticks(code).sort_values('time', ascending=True)
if tmp is not None:
            tmp.loc[:, 'amount'] =  tmp.price * tmp.volume
            tmpagg = tmp.groupby('type').agg({'amount':'sum', 'volume':'sum'})
            bigbuy = tmpagg.loc['买盘', 'amount']  if '买盘' in tmpagg.index else 0
            bigsell = tmpagg.loc['卖盘', 'amount'] if '卖盘' in tmpagg.index else 0
            bigneu = tmpagg.loc['中性盘', 'amount'] if '中性盘' in tmpagg.index else 0
        else:
            bigbuy = 0
            bigsell = 0
            bigneu = 0
        avg_price = round(tt.amount.sum() / tt.volume.sum() / 100, 2)
        net_ratio = (bigbuy-bigsell)/tt.amount.sum()*100
        now_price = tt.loc[:, 'price'].values[-1]
        now_time = pdl.parse(tt.loc[:, 'time'].values[-1])

        idx = pd.IndexSlice
        tick_rt.loc[idx[code, now_time], 'avg_price']=avg_price
        tick_rt.loc[idx[code, now_time], 'net_ratio']=net_ratio
        tick_rt.loc[idx[code, now_time], 'now_price']=now_price
        tick_rt.loc[idx[code, now_time], 'bigbuy']=bigbuy
        tick_rt.loc[idx[code, now_time], 'bigsell']=bigsell
        tick_rt.loc[idx[code, now_time], 'bigneu']=bigneu   

,,volume,amount
ts_code,time,,
002233.SZ,2020-09-22 17:03:49.881658,9926497.0,1.480423e+08
603809.SH,2020-09-22 17:03:49.881658,1687496.0,4.236285e+07
002897.SZ,2020-09-22 17:03:49.881658,1408079.0,4.843900e+07
002906.SZ,2020-09-22 17:03:49.881658,2954159.0,5.120960e+07
002596.SZ,2020-09-22 17:03:49.881658,6310042.0,4.085287e+07
...,...,...,...
603392.SH,2020-09-22 17:03:49.881658,2124774.0,4.000424e+08
601186.SH,2020-09-22 17:03:49.881658,22281704.0,1.938561e+08
603538.SH,2020-09-22 17:03:49.881658,1829782.0,7.334518e+07


In [18]:
price_df

,time,code,open,close,high,low,volume,money,ts_code
0,2020-09-22 16:53:19.397098,002233.XSHE,14.91,14.82,15.05,14.82,9926497.0,1.480423e+08,002233.SZ
1,2020-09-22 16:53:19.397098,603809.XSHG,25.34,24.91,25.67,24.80,1687496.0,4.236285e+07,603809.SH
2,2020-09-22 16:53:19.397098,002897.XSHE,34.36,34.29,34.90,33.90,1408079.0,4.843900e+07,002897.SZ
3,2020-09-22 16:53:19.397098,002906.XSHE,17.02,17.17,17.77,16.91,2954159.0,5.120960e+07,002906.SZ
4,2020-09-22 16:53:19.397098,002596.XSHE,6.47,6.43,6.52,6.43,6310042.0,4.085287e+07,002596.SZ
...,...,...,...,...,...,...,...,...,...
3609,2020-09-22 16:53:19.397098,603392.XSHG,179.00,187.01,193.88,179.00,2124774.0,4.000424e+08,603392.SH
3610,2020-09-22 16:53:19.397098,601186.XSHG,8.73,8.65,8.76,8.65,22281704.0,1.938561e+08,601186.SH
3611,2020-09-22 16:53:19.397098,603538.XSHG,40.18,39.68,40.76,39.50,1829782.0,7.334518e+07,603538.SH
3612,2020-09-22 16:53:19.397098,601598.XSHG,3.89,3.89,3.95,3.83,39926334.0,1.553834e+08,601598.SH


In [25]:
df = df.drop_duplicates()

In [33]:
df = df.groupby(['ts_code', 'kind']).agg({'volume': 'sum', 'amount': 'sum'})
df

volume        amount
ts_code   kind                        
000001.SZ D     51770279  8.254034e+08
          E      1176794  1.864363e+07
          U     69289850  1.103131e+09
000002.SZ D     60975629  1.767269e+09
          E      9206795  2.688821e+08
...                  ...           ...
900917.SH U       588900  2.031705e+05
900926.SH U       112735  3.802552e+05
900932.SH U       509906  4.267913e+05
900933.SH U       537420  1.236066e+06
900948.SH U       760600  5.255746e+05

[10594 rows x 2 columns]

In [33]:
df = df.groupby(['ts_code', 'kind']).agg({'volume': 'sum', 'amount': 'sum'})
df

volume        amount
ts_code   kind                        
000001.SZ D     51770279  8.254034e+08
          E      1176794  1.864363e+07
          U     69289850  1.103131e+09
000002.SZ D     60975629  1.767269e+09
          E      9206795  2.688821e+08
...                  ...           ...
900917.SH U       588900  2.031705e+05
900926.SH U       112735  3.802552e+05
900932.SH U       509906  4.267913e+05
900933.SH U       537420  1.236066e+06
900948.SH U       760600  5.255746e+05

[10594 rows x 2 columns]

In [36]:
tmp = df.reset_index().pivot(index='ts_code', values='amount', columns='kind')
tmp.loc[:,'net_amount'] = tmp.U - tmp.D
tmp_res = 

kind,D,E,U
ts_code,,,
000001.SZ,8.254034e+08,1.864363e+07,1.103131e+09
000002.SZ,1.767269e+09,2.688821e+08,2.455772e+09
000004.SZ,4.558864e+07,7.468401e+06,3.090193e+07
000005.SZ,6.356740e+06,4.647360e+05,4.948897e+06
000006.SZ,1.139013e+08,2.221684e+06,1.964142e+08
...,...,...,...
900917.SH,NaN,NaN,2.031705e+05
900926.SH,NaN,NaN,3.802552e+05
900932.SH,NaN,NaN,4.267913e+05


In [35]:
df.index

MultiIndex([('000001.SZ', 'D'),
            ('000001.SZ', 'E'),
            ('000001.SZ', 'U'),
            ('000002.SZ', 'D'),
            ('000002.SZ', 'E'),
            ('000002.SZ', 'U'),
            ('000004.SZ', 'D'),
            ('000004.SZ', 'E'),
            ('000004.SZ', 'U'),
            ('000005.SZ', 'D'),
            ...
            ('688981.SH', 'D'),
            ('688981.SH', 'E'),
            ('688981.SH', 'U'),
            ('900905.SH', 'U'),
            ('900908.SH', 'U'),
            ('900917.SH', 'U'),
            ('900926.SH', 'U'),
            ('900932.SH', 'U'),
            ('900933.SH', 'U'),
            ('900948.SH', 'U')],
           names=['ts_code', 'kind'], length=10594)

In [56]:
len(stock_list)

3529

,display_name,name,start_date,end_date,type,status,not_new
000001.XSHE,平安银行,PAYX,1991-04-03,2200-01-01,stock,OK,True
000002.XSHE,万科A,WKA,1991-01-29,2200-01-01,stock,OK,True
000004.XSHE,国农科技,GNKJ,1990-12-01,2200-01-01,stock,OK,True
000005.XSHE,世纪星源,SJXY,1990-12-10,2200-01-01,stock,OK,True
000006.XSHE,深振业A,SZYA,1992-04-27,2200-01-01,stock,OK,True
...,...,...,...,...,...,...,...
605358.XSHG,立昂微,LAW,2020-09-11,2200-01-01,stock,OK,False
605366.XSHG,宏柏新材,HBXC,2020-08-12,2200-01-01,stock,OK,False
605369.XSHG,拱东医疗,GDYL,2020-09-16,2200-01-01,stock,OK,False
605388.XSHG,均瑶健康,JYJK,2020-08-18,2200-01-01,stock,OK,False


In [6]:
# 1. dd net_amount
# 2. total_amount current
# 3. tick net_amount change(every minute)

current = 
ticks = 

True

In [45]:
df.sort_values('volume').tail(2).volume.min()

333122280

In [48]:
df.groupby('ts_code').volume.transform(lambda x: x.nsmallest(2).max())

ts_code    kind
000001.SZ  D       51770279
           E       51770279
           U       51770279
000002.SZ  D       60975629
           E       60975629
                     ...   
900917.SH  U         588900
900926.SH  U         112735
900932.SH  U         509906
900933.SH  U         537420
900948.SH  U         760600
Name: volume, Length: 10594, dtype: int64

In [49]:
df

volume        amount
ts_code   kind                        
000001.SZ D     51770279  8.254034e+08
          E      1176794  1.864363e+07
          U     69289850  1.103131e+09
000002.SZ D     60975629  1.767269e+09
          E      9206795  2.688821e+08
...                  ...           ...
900917.SH U       588900  2.031705e+05
900926.SH U       112735  3.802552e+05
900932.SH U       509906  4.267913e+05
900933.SH U       537420  1.236066e+06
900948.SH U       760600  5.255746e+05

[10594 rows x 2 columns]

In [1]:
akcode = add_postfix('002570', type='ak')
tmpdf = ak.stock_zh_a_spot()

NameError: name 'add_postfix' is not defined